In [74]:
import marvin

from typing import List, Tuple
from pydantic import BaseModel
import os
from marvin.beta.assistants import Assistant
from marvin.beta.assistants import Thread
from marvin.beta.assistants.formatting import pprint_messages
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

In [13]:
marvin.settings.openai.api_key = os.environ["OPENAI_API_KEY"]
PERPLEXITY_API_KEY = os["PERPLEXITY_API_KEY"]

client = OpenAI()

In [53]:
ai = Assistant(name="Founder", 
            instructions="You are entrepneur exploring ideas to assess their desirability, viability and feasability.")

In [69]:
thread = Thread()

In [70]:
thread.add('Hello')

Message(id='msg_6oRNSF21NQla7qO0dD4EiQIe', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Hello'), type='text')], created_at=1718819805, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_VD4CeS0eDljXcySK3R6n7y8Y')

In [82]:
thread.add('How is it going?')

Message(id='msg_K0ghpQjK9bF5PKaLswIosSNZ', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='How is it going?'), type='text')], created_at=1718820104, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_VD4CeS0eDljXcySK3R6n7y8Y')

In [83]:
thread.run(ai)

Run(thread=Thread(id='thread_VD4CeS0eDljXcySK3R6n7y8Y', metadata={}), assistant=Assistant(id=None, name='Founder', description=None, instructions='You are entrepneur exploring ideas to assess their desirability, viability and feasability.', model='gpt-4o', tools=[], tool_resources={}, metadata={}), event_handler_class=None, event_handler_kwargs={}, model=None, instructions=None, additional_instructions=None, tools=None, additional_tools=None, data=None)

In [84]:
messages = thread.get_messages()

In [85]:
messages

[Message(id='msg_6oRNSF21NQla7qO0dD4EiQIe', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Hello'), type='text')], created_at=1718819805, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_VD4CeS0eDljXcySK3R6n7y8Y'),
 Message(id='msg_K0ghpQjK9bF5PKaLswIosSNZ', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='How is it going?'), type='text')], created_at=1718820104, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_VD4CeS0eDljXcySK3R6n7y8Y'),
 Message(id='msg_vlgi5FPhavr0LQNtQnWB3eHJ', assistant_id='asst_CjZaqlK8Cb8m4rGSnetv8U33', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="Hello! I'm doing well, thank you. How can I assist you today?

In [27]:
class Question(BaseModel):
    text: str
    reasoning: str

class QA(BaseModel):
    question: Question
    answer: str

class Followup(BaseModel):
    qa: QA
    next_q: Question

In [45]:
topic = "Payment API providers for businesses in Canada. Emphasis on Canada."

prompt_initial = f"""
Given the following topic, generate a list of crutial initial questions to start your research. 
Limit yourself to a maximum of 5 questions.
As a founder, choose the most important questions to ask about this topic, in order to decide whether
there's a worthy business opportunity here.
For each question, also explain your reasoning for including it.
Topic: {topic}
"""

In [46]:
system_content = """
You are an entrepreneur looking to do market research on a given vertical.
Your ultimate goal is to find ideas that are desirable (do people have this problem and want a solution for it),
feasible (it's possible to build this solution) and viable (there's a business model that makes sense).
You also want to review existing solutions in the market and you want to make sure you have found a gap,
i.e. a problem that is not well solved by current solutions.
Finally, you want to be able to point to the target segment for this problem/solution.
"""

In [47]:
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": ""},
    {"role": "user", "content": prompt_initial}
  ]
)

r = completion.choices[0].message

In [48]:
r

ChatCompletionMessage(content='1. **What are the key features and services offered by existing payment API providers in Canada?**\n   - **Reasoning:** Understanding the current market offerings helps identify what is already available and can highlight any unique requirements or gaps that new solutions can fill. \n\n2. **Who are the primary competitors in the Canadian payment API market, and what are their strengths and weaknesses?**\n   - **Reasoning:** Knowing your competition is crucial for positioning your service. Analyzing their strengths and weaknesses provides insights into areas for improvement or differentiation.\n\n3. **What are the legal and regulatory requirements for payment API providers operating in Canada?**\n   - **Reasoning:** Compliance with financial regulations is essential for operating in the payment space. Understanding legal requirements upfront can help in planning and avoiding costly legal issues later.\n\n4. **What are the specific needs and pain points of 

In [49]:
analysis_plan = marvin.extract(r, target=Question)

In [52]:
for q in analysis_plan:
    print('Question:')
    print(q.text)
    print('Reasoning:')
    print(q.reasoning)
    print()

Question:
What are the key features and services offered by existing payment API providers in Canada?
Reasoning:
Understanding the current market offerings helps identify what is already available and can highlight any unique requirements or gaps that new solutions can fill.

Question:
Who are the primary competitors in the Canadian payment API market, and what are their strengths and weaknesses?
Reasoning:
Knowing your competition is crucial for positioning your service. Analyzing their strengths and weaknesses provides insights into areas for improvement or differentiation.

Question:
What are the legal and regulatory requirements for payment API providers operating in Canada?
Reasoning:
Compliance with financial regulations is essential for operating in the payment space. Understanding legal requirements upfront can help in planning and avoiding costly legal issues later.

Question:
What are the specific needs and pain points of businesses in Canada when it comes to payment processi

In [ ]:
docket = []

for q in analysis_plan:
    # answer question + extract output as QA object
    perplexity_prompt = """
    {q.question}
    """
    # perplexity api call -> answer

    answer_str = """
    Question: {q.question}
    Reasoning: {q.reasoning}
    Answer: {answer}
    """

    qa = marvin.extract(answer_str, target=QA)
    docket.append(qa)

In [ ]:
prompt_final_questions = ""

for idx, qa in enumerate(docket):
    prompt_final_questions += str(idx) + '.\n'
    prompt_final_questions += 'Question: ' + qa.question.text + '\n'
    prompt_final_questions += 'Reasoning: ' + qa.question.reasoning + '\n'
    prompt_final_questions += 'Answer: ' + qa.answer + '\n'
    prompt_final_questions += '\n'

In [ ]:
prompt_final_base = """
Here are the questions you want to be able to answer:

a. What is the market need?

Explanation: To understand a market need is to understand a (currently) unfilled desire or potential. 
New ventures generally begin with a hypothesis that this potential is real, 
but the first order of business is always to validate this assumption, 
and usually to revise the hypothesis on the basis of facts and evidence.

b. What is a feasible solution?

Explanation: The entrepreneur will try to understand what will be required to offer a compelling solution 
that will actually spur the change in consumption required (solve the market need).
 This discipline will minimize that chance of creating a product concept that misses the mark or 
 fails to take into account critical dimensions of the need or possible inhibitors to adoption.

c. What is the competition

Explanation: Competition is the universe of alternatives that the potential customer has. 
Competition must be considered from the customers perspective. 
What are the full set of alternatives that the customer has.

To answer these questions, here's the docket you have assembled so far:

{prompt_final_questions}

Do you have all the information you need to answer questions a, b and c?
"""